In [1]:
!pip install requests

In [2]:
# Add your OpenWeatherMap API key here
WEATHER_API_KEY = "cf2d3f6d2796231d42929051d92f9581"

In [4]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, AIMessage

class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "the messages in the conversation"]
    city: str
    interests: List[str]
    itinerary: str
    num_days: int
    companions: str
    budget: int
    dates: List[str]  # New field to store trip dates

In [5]:
def input_trip_dates(state: PlannerState) -> PlannerState:
    print("Enter the start date of your trip (YYYY-MM-DD): ")
    start_date = input("Start Date: ")
    state["dates"] = [start_date]

    # Calculate end date based on number of days
    from datetime import datetime, timedelta
    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    for i in range(1, state["num_days"]):
        next_date = start_date_obj + timedelta(days=i)
        state["dates"].append(next_date.strftime("%Y-%m-%d"))

    state["messages"].append(HumanMessage(content=f"Trip Dates: {', '.join(state['dates'])}"))
    return state

In [6]:
import requests

def get_weather(city: str, date: str, api_key: str) -> str:
    """
    Fetch weather data for a specific city and date using OpenWeatherMap API.
    """
    base_url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric",  # Use "imperial" for Fahrenheit
        "cnt": 40  # Number of forecasts (5 days * 8 forecasts per day)
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        forecasts = response.json()["list"]
        for forecast in forecasts:
            forecast_date = forecast["dt_txt"].split()[0]  # Extract date part
            if forecast_date == date:
                weather_description = forecast["weather"][0]["description"]
                temperature = forecast["main"]["temp"]
                humidity = forecast["main"]["humidity"]
                return f"Weather on {date}: {weather_description}, Temperature: {temperature}°C, Humidity: {humidity}%"
        return f"Weather data not available for {date}."
    else:
        return "Unable to fetch weather data."

In [22]:
from langchain_groq import ChatGroq

# Initialize the language model
llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name="llama-3.3-70b-versatile"
)

In [23]:
# Function to input the number of days
def input_num_days(state: PlannerState) -> PlannerState:
    state["num_days"] = int(input("Enter the number of days for your trip: "))
    state["messages"].append(HumanMessage(content=f"Number of Days: {state['num_days']}"))
    return state

# Function to input the city
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your trip: ")
    user_message = input("Your Input: ")
    state["city"] = user_message
    state["messages"].append(HumanMessage(content=user_message))
    return state

# Function to input interests
def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter your interests for the trip to {state['city']} (comma-separated): ")
    user_message = input("Your Input: ")
    state["interests"] = [interest.strip() for interest in user_message.split(",")]
    state["messages"].append(HumanMessage(content=user_message))
    return state

# Function to input trip details (companions and budget)
def input_trip_details(state: PlannerState) -> PlannerState:
    state["companions"] = input("Enter your travel companions (e.g., solo, family, friends): ")
    state["budget"] = int(input("Enter your total budget for the trip (in USD): "))
    state["messages"].append(HumanMessage(content=f"Trip Details - Companions: {state['companions']}, Budget: ${state['budget']}"))
    return state

# Function to input trip dates
def input_trip_dates(state: PlannerState) -> PlannerState:
    print("Enter the start date of your trip (YYYY-MM-DD): ")
    start_date = input("Start Date: ")
    state["dates"] = [start_date]

    # Calculate end date based on number of days
    from datetime import datetime, timedelta
    start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
    for i in range(1, state["num_days"]):
        next_date = start_date_obj + timedelta(days=i)
        state["dates"].append(next_date.strftime("%Y-%m-%d"))

    state["messages"].append(HumanMessage(content=f"Trip Dates: {', '.join(state['dates'])}"))
    return state

# Function to create the itinerary
def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"Creating a {state['num_days']}-day itinerary for {state['city']} based on interests: {', '.join(state['interests'])}")

    # Fetch weather for each day
    weather_info = []
    for date in state["dates"]:
        weather = get_weather(state["city"], date, WEATHER_API_KEY)
        weather_info.append(weather)

    # Generate itinerary
    response = llm.invoke(itinerary_prompt.format_messages(
        city=state['city'],
        interests=', '.join(state['interests']),
        num_days=state['num_days'],
        companions=state['companions'],
        budget=state['budget'],
        weather_info='\n'.join(weather_info)  # Pass weather info to the prompt
    ))

    print("\nFinal Itinerary: ")
    print(response.content)
    state["messages"].append(AIMessage(content=response.content))
    state["itinerary"] = response.content
    return state

In [10]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00


In [12]:
import langgraph

In [13]:
!pip install langchain langchain_core langchain_groq langchain_community langgraph requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [17]:
from langgraph.graph import StateGraph, END

# Define the state graph workflow
workflow = StateGraph(state_schema=PlannerState)

# Add nodes to the workflow
workflow.add_node("input_num_days", input_num_days)
workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("input_trip_details", input_trip_details)
workflow.add_node("input_trip_dates", input_trip_dates)  # New node
workflow.add_node("create_itinerary", create_itinerary)

# Set the entry point
workflow.set_entry_point("input_num_days")

# Add edges to define the flow
workflow.add_edge("input_num_days", "input_city")
workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "input_trip_details")
workflow.add_edge("input_trip_details", "input_trip_dates")  # Connect to new node
workflow.add_edge("input_trip_dates", "create_itinerary")
workflow.add_edge("create_itinerary", END)

# Compile the workflow
app = workflow.compile()

In [19]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

In [20]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day-to-day {num_days}-day trip itinerary for {city} based on the user's interests: {interests}. The user is traveling with {companions} and has a budget of {budget} USD. Include recommendations for hotels, food stalls, and activities. Provide a detailed, bulleted itinerary for each day. Here is the weather forecast for each day:\n{weather_info}"),
    ("human", "Create an itinerary for my trip."),
])

In [24]:
def travel_planner():
    print("Welcome to the Travel Planner!")

    # Initialize the state
    state = {
        "messages": [],
        "city": "",
        "interests": [],
        "num_days": 0,
        "companions": "",
        "budget": 0,
        "itinerary": "",
        "dates": []  # Initialize dates
    }

    # Stream the workflow
    for output in app.stream(state):
        pass

# Run the travel planner
travel_planner()

Welcome to the Travel Planner!
Enter the number of days for your trip: 2
Please enter the city you want to visit for your trip: 
Your Input: Mumbai
Please enter your interests for the trip to Mumbai (comma-separated): 
Your Input: Food, Movies
Enter your travel companions (e.g., solo, family, friends): solo
Enter your total budget for the trip (in USD): 400
Enter the start date of your trip (YYYY-MM-DD): 
Start Date: 2025-03-04
Creating a 2-day itinerary for Mumbai based on interests: Food, Movies

Final Itinerary: 
Given your interests in food and movies, I've created a 2-day itinerary for your solo trip to Mumbai with a budget of $400. Since the weather forecast is unavailable, I'll provide a general plan that can be adjusted according to the weather conditions during your trip.

### Day 1: Exploring Mumbai's Culinary Delights and Bollywood

* **Morning:**
  - Start your day with a delicious breakfast at **Cafe Madras** (approximately $5-$7) in Matunga, known for its South Indian cui